In [1]:
import pandas as pd

## CSV读取

In [ ]:
ratings = pd.read_csv('../data/ml-25ms/ratings.csv')
ratings.head()

In [ ]:
user_item_counts = ratings.groupby('userId')['movieId'].nunique().reset_index()
user_item_counts.columns = ['userId', 'item_count']
user_item_counts['item_count'].min(), user_item_counts['item_count'].max()

In [ ]:
ratings['rating'].min(), ratings['rating'].max()

In [ ]:
movies = pd.read_csv('../data/ml-25m/movies.csv')
movies.head()

In [ ]:
item_entities = movies.iloc[:, :2].values.tolist()  # 只取前两列
item_entities = [[str(x), str(y)] for x, y in item_entities]
item_entities[:10]

## dat读取

In [2]:
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])
ratings.head()

C:\Users\dell\AppData\Local\Temp\ipykernel_26436\874908152.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 按照userid和时间戳进行排序
ratings = ratings.sort_values(['userId', 'timestamp'])
ratings.reset_index(drop=True, inplace=True)
# 查找userid=2547的用户记录
df = ratings[(ratings['userId'] == 1) & (ratings['rating'] > 2)]
df = df.reset_index(drop=True)
df

,userId,movieId,rating,timestamp
0,1,3186,4,978300019
1,1,1270,5,978300055
2,1,1721,4,978300055
3,1,1022,5,978300055
4,1,2340,3,978300103
5,1,1836,5,978300172
6,1,3408,4,978300275
7,1,2804,5,978300719
8,1,1207,4,978300719
9,1,1193,5,978300760


In [4]:
movieids = df['movieId'].values.tolist()
1924 in movieids, movieids[:movieids.index(1924)]

ValueError: 1924 is not in list

In [ ]:
# 找到movieId=3634的行索引
i = df[df['movieId']==3634].index[0]
i

In [ ]:
# 筛选出索引大于等于i的行
df = df.iloc[i:]

In [ ]:
# 计算每个用户的评分次数
user_rating_counts = ratings.groupby('movieId').size().reset_index(name='rating_count')
# 找出最小评分次数
min_rating_count = user_rating_counts['rating_count'].min()
print(f"用户评分的最小次数: {min_rating_count}")
print(f"用户评分的最小次数: {user_rating_counts['rating_count'].max()}")

In [ ]:
# 查看userid=1的用户按时间戳排序的交互记录
ratings[ratings['userId']==10].sort_values('timestamp', ascending=False)

In [ ]:
ratings['userId'].max(), ratings['userId'].min()

In [ ]:
movies = pd.read_csv('../data/ml-10M100K/movies.dat', sep='::', names=['movieId', 'title', 'genres'])
movies.head()

In [ ]:
ratings[ratings['rating'] > 2]

In [ ]:
from collections import Counter

df = ratings.sort_values(['userId', 'timestamp'])  # sort for user and time

# 删除评分小于 score 的交互
df = df[df['rating'] > 2]

# 获取用户物品交互列表
user_item = df[['userId', 'movieId']].values.tolist()
user_item = [list(map(str, line)) for line in user_item]

users, items = zip(*user_item)
user_list = []
item_list = []

users_cnt = Counter(users)
for user, count in users_cnt.items():
    if count >= 10:
        user_list.append(user)

items_cnt = Counter(items)
for item, count in items_cnt.items():
    if count >= 10:
        item_list.append(item)

num_user, num_item = len(user_list), len(item_list)

In [ ]:
len(user_item), len(item_list)

In [ ]:
user2id, item2id = reverse_and_map(user_list), reverse_and_map(item_list)
len(user2id), len(item2id)

In [ ]:
min(item2id.keys()), max(item2id.keys())

In [ ]:
res = []
for x, y in user_item:
    if x in user2id.keys() and y in item2id.keys():
        res.append([user2id[x], item2id[y]])
    else:
        print(x, y)
        print(x in user2id.keys())
        print(y in item2id.keys())
        break

In [ ]:
len(res)